In [1]:
suppressWarnings(suppressPackageStartupMessages({
    library(tidyverse)
    library(data.table)
}))


In [7]:
# input file/dir
cascade_d <- '/oak/stanford/groups/mrivas/users/ytanigaw/repos/rivas-lab/biomarkers/cascade/out_v3'
gwas_catalog_filtered <- 'GWAS-catalog-dump/gwas-association-downloaded_2020-08-18.filtered.traits.tsv'

# output file
out_f <- 'fuji.input.tsv'


In [40]:
gwas_catalog_df <- fread(
    gwas_catalog_filtered,
    select=c(
        'CHR_ID', 'CHR_POS', 'SNPS', 
        'P-VALUE', 'OR or BETA', 
        'PUBMEDID', 'DISEASE/TRAIT', 
        'MAPPED_TRAIT', 'MAPPED_TRAIT_URI'
    )
) %>%
rename('CHROM'='CHR_ID', 'POS'='CHR_POS', 'P_EBI'='P-VALUE')

In [58]:
gwas_catalog_df %>% select(SNPS) %>% unique() %>% dim() %>% print()
gwas_catalog_df %>% select(CHROM, POS, SNPS) %>% unique() %>% dim() %>% print()

[1] 968   1
[1] 968   3


In [41]:
dfs <- list()
for(vtype in c('PTVs', 'PAVs')){
    dfs[[vtype]] <- fread(
        file.path(cascade_d, sprintf('cascade.%s.tsv', vtype)),
        colClasses = c('#CHROM'='character')
    ) %>% rename('CHROM'='#CHROM')
}


In [42]:
dfs[['PTVs']] %>%
dim()

[1] 58 23

In [43]:
dfs[['PAVs']] %>%
dim()

[1] 1323   23

In [75]:
dfs[['PTVs']] %>%
select(CHROM, POS, REF, ALT, ID, trait, BETA, SE, P, Gene_symbol) %>%
left_join(
    gwas_catalog_df %>% select(CHROM, POS) %>%
    mutate(has_EBI = T)
) %>%
replace_na(list(has_EBI=F)) %>%
filter(has_EBI)

Joining, by = c("CHROM", "POS")



CHROM,POS,REF,ALT,ID,trait,BETA,SE,P,Gene_symbol,has_EBI
<chr>,<int>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<lgl>


In [76]:
dfs[['PAVs']] %>%
select(CHROM, POS, REF, ALT, ID, trait, BETA, SE, P, Gene_symbol) %>%
left_join(
    gwas_catalog_df %>% select(CHROM, POS) %>%
    mutate(has_EBI = T)
) %>%
replace_na(list(has_EBI=F)) %>%
filter(has_EBI)

Joining, by = c("CHROM", "POS")



CHROM,POS,REF,ALT,ID,trait,BETA,SE,P,Gene_symbol,has_EBI
<chr>,<int>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<lgl>
